In [2]:
# Importando bibliotecas
!pip -q install google-genai

In [3]:
# Configurando a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [23]:
# Configurando o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [5]:
# Instalando o Framework de agentes do Google
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [6]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [7]:
# ----- FUNÇÕES AUXILIARES -----
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [21]:
##########################################
# --- Agente 1: Curador de recursos --- #
##########################################

def agente_curador_recursos(topico)
    agente_curador = Agent(
        name="agente_curador",
        model= MODEL_ID,
        instruction="""
        Você é um especialista em encontrar os melhores e mais confiáveis recursos online para estudo.
        Sua tarefa é receber um tópico de estudo do usuário e, usando a ferramenta de busca do Google (Google_search),
        encontrar e listar 3 a 5 links relevantes para artigos, vídeos, tutoriais ou explicações
        que ajudem o usuário a aprender sobre o tópico.

        **Priorize fontes de alta qualidade e autoridade no assunto.
        ** Procure por:
        * Artigos acadêmicos ou de pesquisa: De universidades, institutos de pesquisa ou periódicos científicos.
        * Sites de instituições de ensino: Universidades, escolas, plataformas educacionais renomadas.
        * Livros didáticos ou e-books reconhecidos: Se puder encontrar links para acesso ou informações sobre eles.
        * Canais educativos renomados no YouTube ou outras plataformas: Canais de professores, universidades, ou portais de ciência/educação.
        * **Evite:** Blogs pessoais sem referência, fóruns de discussão (a menos que sejam de especialistas), sites de notícias generalistas (a menos que citem fontes primárias).

        Formate sua resposta como uma lista numerada de links, cada um com uma breve descrição do que o recurso oferece.
        Exemplo:
        1. [Título do Artigo] - Breve descrição do conteúdo (URL do Artigo)
        2. [Título do Vídeo] - Explica sobre X assunto (URL do Vídeo)
        """,
        description="Agente que encontra e lista recursos online (artigos, vídeos, tutoriais) sobre um tópico de estudo, priorizando fontes confiáveis.",
        tools=[google_search]
    )
    entrada_agente_curador = f"Tópico que quero estudar: {topico}"
    recursos_encontrados_string = call_agent(agente_curador, entrada_agente_curador)
    return recursos_encontrados_string

In [25]:
######################################
# --- Agente 2: Planejador de foco --- #
######################################

def agente_planejador_foco(tempo_total, recursos_encontrados_string):
    agente_planejador = Agent(
        name="agente_planejador_foco",
        model= MODEL_ID,
        instruction="""
        Você é um mestre da produtividade e um planejador de sessões de estudo.
        Sua tarefa é receber uma lista de recursos de estudo (links e descrições)
        e o tempo total disponível do usuário para estudar (ex: "2 horas", "45 minutos").

        Com base nesses dados, crie um plano de estudo detalhado, dividindo o tempo em "sprints" focadas
        (sugestão: 25 minutos de estudo intenso + 5 minutos de pausa, estilo Pomodoro),
        alocando os recursos para cada sprint. Tente distribuir os recursos de forma lógica e
        sugira o que o usuário deve fazer em cada sprint (ex: "Sprint 1: Ler o Artigo X").

         **Ao planejar, considere a duração estimada dos recursos, como vídeos, e aloque tempo suficiente para eles nas sprints.
         **Se um vídeo tem 15 minutos, por exemplo, ele deve ocupar uma parte proporcional da sprint.

        Formate o plano de forma clara, usando cabeçalhos para cada sprint e bullet points.
        Adicione um toque motivacional ou um pequeno "desafio" no final do plano para encorajar o usuário.

        Exemplo de formato:
        --- Plano de Estudo Focado ---
        Tempo Total Disponível: [Tempo do Usuário]

        Sprint 1 (25 min de estudo + 5 min de pausa):
        * Objetivo: Ler o Artigo sobre [Assunto] (link: https://pt.wikipedia.org/wiki/Artigo_%28gram%C3%A1tica%29)
        * Foco: Compreender os conceitos básicos de...

        Sprint 2 (25 min de estudo + 5 min de pausa):
        * Objetivo: Assistir o vídeo sobre [Assunto] (link: https://www.youtube.com/letsdovideo)
        * Foco: Anotar pontos chave e exemplos.

        [... outras sprints ...]


        Acrescente um Desafio Bônus ao final e uma frase encorajadora.
        Por exemplo:
        Desafio Bônus: Ao final, tente explicar o conceito principal para si mesmo em voz alta!
        Não desanime! É aos poucos que o conhecimento é construído! 💪
        """,
        description="Agente que cria planos de estudo focados em 'sprints' com base em recursos e tempo disponível.",
        tools=[]
        )

    entrada_agente_planejador = f""" Recursos de estudo: {recursos_encontrados_string}
    Tempo total disponível: {tempo_total}
    """
    sprints = call_agent(agente_planejador, entrada_agente_planejador)
    return sprints


In [26]:
# ---- LÓGICA DO SISTEMA DE AGENTES ----

print("Olá! Vamos criar seu Guia de Estudo Focado. 🎯📚")

topico = input("Qual tópico você gostaria de estudar hoje? (ex: 'Python para iniciantes'): ")
tempo_total = input("Quanto tempo você tem para estudar hoje? (ex: '2 horas', '1 hora e 45 minutos'): ")


if not topico:
    print("Você esqueceu de digitar o tópico! Por favor, tente novamente.")
elif not tempo_total:
    print("Você esqueceu de digitar o tempo de estudo! Por favor, tente novamente.")
else:
    print(f"\nÓtimo! Vamos buscar recursos online sobre '{topico}' e criar um plano de {tempo_total} para te auxiliar nos estudos!")

    try:
        recursos_encontrados_string = agente_curador_recursos(topico)

        print("\n--- 📚 Aqui estão os recursos que encontrei para você: ---\n")
        display(to_markdown(recursos_encontrados_string))
        print("--------------------------------------------------------------")

        # Pequena validação: Se não houver recursos, não faz sentido gerar o plano
        if not recursos_encontrados_string or len(recursos_encontrados_string.strip()) < 20:
            print("\n⚠️ Não foi possível encontrar recursos suficientes. O plano de estudo não será gerado.")
            print("Tente um tópico de estudo diferente ou verifique sua API Key/conexão.")
        else:
            print(f"\nGerando seu plano de estudo focado para {tempo_total}...")

            # Chamando o Agente Planejador de Foco
            plano_final_string = agente_planejador_foco(tempo_total, recursos_encontrados_string)

            print("\n--- Seu Plano de Estudo Focado ---")
            display(to_markdown(plano_final_string))
            print("--------------------------------------------------------------")

        print("\n--- Processo Concluído! Bons estudos! ✨ ---")

    except Exception as e:
        print(f"\n⛔ Ocorreu um erro inesperado: {e}")
        print("Isso pode ser um problema com a API, sua conexão ou nas instruções dos agentes. Tente novamente.")




Olá! Vamos criar seu Guia de Estudo Focado. 🎯📚
Qual tópico você gostaria de estudar hoje? (ex: 'Python para iniciantes'): Agentes de IA
Quanto tempo você tem para estudar hoje? (ex: '2 horas', '1 hora e 45 minutos'): 2 horas e 40 minutos

Ótimo! Vamos buscar recursos online sobre 'Agentes de IA' e criar um plano de 2 horas e 40 minutos para te auxiliar nos estudos!

--- 📚 Aqui estão os recursos que encontrei para você: ---



> Para te ajudar a estudar sobre Agentes de IA, procurei por recursos confiáveis e relevantes. Aqui estão algumas sugestões:
> 
> 
> Com base na minha pesquisa, aqui estão alguns recursos que podem te ajudar a estudar sobre Agentes de IA:
> 
> 1.  **Agentes de IA - Princípios e Fundamentos - Udemy:** Este curso na Udemy oferece uma introdução rápida e direta aos Agentes de IA, abordando seu funcionamento, arquitetura e impacto no futuro do trabalho e das organizações. \[1]
> 2.  **Tutorial Prático - Aprenda a Criar Agente de IA em 2025 - YouTube:** Neste vídeo no YouTube, você aprenderá passo a passo como criar seu primeiro Agente de Inteligência Artificial, desde os conceitos básicos até a otimização do desempenho do seu agente. \[4]
> 3.  **O que são AGENTES de IA: APRENDA do ZERO! - YouTube:** Este vídeo no YouTube explica o conceito de Agentes de IA do zero, diferenciando-os de outras IAs e mostrando como diversas plataformas possibilitam sua criação. \[6]
> 4.  **Curso Agentes IA Gratuito para Iniciantes 2025 | Do Zero ao Agente IA - YouTube:** Este curso gratuito no YouTube ensina como criar um agente inteligente que organiza e gerencia seus eventos de forma automatizada, utilizando ferramentas como n8n e Telegram. \[14]
> 5.  **Artigos sobre agentes de IA | DataCamp:** Explore como as estruturas de agentes de IA permitem fluxos de trabalho autônomos, desde configurações de agente único até orquestração complexa de vários agentes. \[19]
> 
> Espero que esses recursos te ajudem a começar seus estudos sobre Agentes de IA!
> 


--------------------------------------------------------------

Gerando seu plano de estudo focado para 2 horas e 40 minutos...

--- Seu Plano de Estudo Focado ---


> --- Plano de Estudo Focado em Agentes de IA ---
> Tempo Total Disponível: 2 horas e 40 minutos
> 
> Sprint 1 (25 min de estudo + 5 min de pausa):
> * Objetivo: Iniciar com o curso "Agentes de IA - Princípios e Fundamentos" na Udemy.
> * Foco: Absorver os conceitos básicos e entender a arquitetura dos Agentes de IA.
> 
> Sprint 2 (25 min de estudo + 5 min de pausa):
> * Objetivo: Continuar o curso da Udemy.
> * Foco: Identificar o impacto dos Agentes de IA no futuro do trabalho e das organizações.
> 
> Sprint 3 (25 min de estudo + 5 min de pausa):
> * Objetivo: Assistir ao vídeo "O que são AGENTES de IA: APRENDA do ZERO!" no YouTube.
> * Foco: Diferenciar Agentes de IA de outras IAs e entender as plataformas para sua criação.
> 
> Sprint 4 (25 min de estudo + 5 min de pausa):
> * Objetivo: Explorar o tutorial "Aprenda a Criar Agente de IA em 2025" no YouTube.
> * Foco: Anotar os passos práticos para criar um Agente de IA e entender a otimização do desempenho.
> 
> Sprint 5 (25 min de estudo + 5 min de pausa):
> * Objetivo: Começar o curso gratuito "Curso Agentes IA Gratuito para Iniciantes 2025" no YouTube.
> * Foco: Aprender a criar um agente inteligente para organizar eventos usando n8n e Telegram.
> 
> Sprint 6 (25 min de estudo + 5 min de pausa):
> * Objetivo: Continuar o curso gratuito no YouTube.
> * Foco: Aprofundar o conhecimento nas ferramentas n8n e Telegram para automação.
> 
> Desafio Bônus: Após cada vídeo, tente resumir os principais pontos em poucas frases. Isso ajudará na fixação do conteúdo!
> Não desanime! Cada passo te aproxima do domínio dos Agentes de IA! 💪
> 


--------------------------------------------------------------

--- Processo Concluído! Bons estudos! ✨ ---
